# Script SAE15 - Traiter des données




---


Nom :

Prénom :

Groupe de TP :


---



In [ ]:
####################################################################
# DEBUT DE CODE : SAE15 - TRAITER DES DONNEES                      #
#------------------------------------------------------------------#
# PARTIE 1 : IMPORTATIONS, INSTALLATIONS ET CONFIGURATIONS         #
# PARTIE 2 : RECUPERATION DES DONNEES VELIB EN OPENDATA            #
# PARTIE 3 : MISE SOUS FORME DE DATAFRAME                          #
# PARTIE 4 : ANALYSES STATISTIQUES                                 #
# PARTIE 5 : EXPORTATION DES MESURES STATISTIQUES POUR LE WEB      #
# PARTIE 6 : SPATIALISATION DES DONNEES, EXPORTATION POUR LE WEB   #
####################################################################

In [33]:
####################################################################
# PARTIE 1 : IMPORTATIONS, INSTALLATIONS ET CONFIGURATION          #
####################################################################
# modules natifs à importer
!pip install geopandas
!pip install contextily
#-------------------------------------------------------------------
# modules externes à installer et à importer
import sys
import urllib
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
#-------------------------------------------------------------------
# montage du drive
from google.colab import drive
#-------------------------------------------------------------------
# modules internes à importer
import json                    
import pandas as pd
####################################################################

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
####################################################################
# PARTIE 2 : RECUPERATION DES DONNEES VELIB EN OPENDATA            #
####################################################################
# récupération des données statiques (données d'information)
data_info = json.loads(urllib.request.urlopen('https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_information.json').read()) 
#-------------------------------------------------------------------
# récupération des données dynamiques (données de status)
data_status = json.loads(urllib.request.urlopen('https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_status.json').read())
####################################################################

In [35]:
####################################################################
# PARTIE 3 : MISE SOUS FORME DE DATAFRAME                          #
####################################################################
# dataframe de l'information pour les stations
data_information = pd.DataFrame(data_info)
#-------------------------------------------------------------------
# dataframe du status pour les stations
datastatus = pd.DataFrame(data_status)
#-------------------------------------------------------------------
# fusion des deux DataFrames en un seul (merge)
m = pd.merge(pd.DataFrame(datastatus['data']['stations']), pd.DataFrame(data_information['data']['stations'])) 
####################################################################

In [36]:
####################################################################
# PARTIE 4 : ANALYSES STATISTIQUES                                 #
####################################################################
# mesures statistiques sur la capacité des stations
mstat = m['capacity']
#-------------------------------------------------------------------
# mesures statistiques sur les vélos disponibles
mstatvelo = m['numBikesAvailable']
#-------------------------------------------------------------------
# mesures statistiques sur les stands disponibles
mstatstand = m['numDocksAvailable']
#-------------------------------------------------------------------
# mesures statistiques sur les vélos mécaniques (optionnel)

#-------------------------------------------------------------------
# mesures statistiques sur les vélos électriques (optionnel)

#-------------------------------------------------------------------
# assemblage des toutes les statistiques dans un seul DataFrame
merge =  pd.concat([pd.DataFrame(mstat), pd.DataFrame(mstatvelo), pd.DataFrame(mstatstand)], axis=1)

#-------------------------------------------------------------------
# affichage dans la console du DataFrame des statistiques assemblées
display(merge)
####################################################################

,capacity,numBikesAvailable,numDocksAvailable
0,35,7,27
1,30,12,13
2,20,2,16
3,21,3,18
4,30,11,19
...,...,...,...
1444,35,6,28
1445,35,7,27
1446,35,8,26
1447,25,21,6


In [37]:
####################################################################
# PARTIE 5 : EXPORTATION DES MESURES STATISTIQUES POUR LE WEB      #
####################################################################
# mise en forme du DataFrame des statistiques avant exportation
stat = merge.rename(columns = {"lat":"1", "lon":"2", "capacity":"3", "station_id":"4", "num_bikes_available":"5", "numBikesAvailable":"6", "num_docks_available":"7", "numDocksAvailable":"8", "is_installed":"9", "is_returning":"A", "is_renting":"B","last_reported":"16"})
#-------------------------------------------------------------------
# exportation du DataFrame des statistiques au format HTML
stat.to_html()
####################################################################

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>3</th>\n      <th>6</th>\n      <th>8</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>35</td>\n      <td>7</td>\n      <td>27</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>30</td>\n      <td>12</td>\n      <td>13</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>20</td>\n      <td>2</td>\n      <td>16</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>21</td>\n      <td>3</td>\n      <td>18</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>30</td>\n      <td>11</td>\n      <td>19</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>25</td>\n      <td>2</td>\n      <td>23</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>60</td>\n      <td>19</td>\n      <td>39</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>22</td>\n      <td>7</td>\n      <td>14</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>12</td>

In [39]:
####################################################################
# PARTIE 6 : SPATIALISATION DES DONNEES, EXPORTATION POUR LE WEB   #
####################################################################
# transformation du DataFrame des stations Vélib en GeoDataFrame
geom = gpd.points_from_xy(m["lon"], m["lat"])
geo_data = gpd.GeoDataFrame(m, crs="EPSG:4326", geometry=geom)
#-------------------------------------------------------------------
# détermination de la dernière date de mise à jour des données 

#-------------------------------------------------------------------
# carte du taux de disponibilité des vélos avec date de mise à jour 
# et export web de la carte
geo_data_with_map = geo_data.to_crs(epsg=3857)

f, axes = plt.subplots(1, figsize=(17,17))

geo_data_with_map.plot(geo_data.num_bikes_available, markersize=6*geo_data.num_bikes_available, cmap="Paired", ax=axes)

axes.set_axis_off()

ctx.add_basemap(axes)

#plt.savefig('/content/drive/My Drive/Colab notebooks/SAE15/web/data/carte1.svg')

plt.show()
#-------------------------------------------------------------------
# carte du taux de disponibilité des stands avec date de mise à jour
# et export web de la carte
geo_data_with_map = geo_data.to_crs(epsg=3857)

f, axes = plt.subplots(1, figsize=(17,17))

geo_data_with_map.plot(geo_data.num_docks_available, markersize=6*geo_data.num_docks_available, cmap="Paired", ax=axes)

axes.set_axis_off()

ctx.add_basemap(axes)

#plt.savefig('/content/drive/My Drive/Colab notebooks/SAE15/web/data/carte1.svg')

plt.show()
#-------------------------------------------------------------------
# carte du taux de disponibilité des vélos mécaniques (optionnel) 
# et export web de la carte

#-------------------------------------------------------------------
# carte du taux de disponibilité des vélos électriques (optionnel) 
# et export web de la carte

####################################################################

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
####################################################################
# FIN DE CODE : VERIFIER LA PUBLICATION SUR LE WEB                 #
####################################################################